In [82]:
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
import re
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import csv

In [2]:
# Funcion para convertir el contenido de un pdf a txt
def getPDFFileContentToTXT(pdfFile):
    myPDFFile = PyPDF2.PdfFileReader(pdfFile)
    
#Creamos un archivo txt con la informacion del pdf 
    with open('pdfContenido.txt', 'w') as pdf_output:
        for page in range (myPDFFile.getNumPages()):
            data = myPDFFile.getPage(page).extractText()
            pdf_output.write(data)

# Leemos la infromacion que acabamos de escribir e quitamos los espacios de lineas
    with open('pdfContenido.txt', 'r') as myPDFContent:
        return myPDFContent.read().replace('\n',' ')

In [3]:
def vacanteCSV_str(file,vacante): #requiere el arcvhivo csv de las vacantes y la vacante que se analizara
    df = pd.read_csv(file)
    df = df.fillna(" ") #rellenamos las casillas sin nada con un espacio
    requerimientos = ""
    
    for i in df[vacante]:
        requerimientos = requerimientos +i + " "
    
    return requerimientos

In [47]:
def tokenize1_2_3_4(pdfFileContent,requerimientos):
    
    count = 1
    for letter in range(len(pdfFileContent)): #funcion para limpiar y corregir errores de transformacion a txt
    
        count +=1
        if count == len(pdfFileContent):
            break
    
        l1 = pdfFileContent[letter]
        l2 = pdfFileContent[letter + 1]
        l3 = pdfFileContent[letter + 2]
    
        if l1.islower() == True and l2.isupper() == True and l3.islower() == True: #Errores de \n
            pdfFileContent = pdfFileContent.replace(l1+l2+l3,l1+" "+l2+l3)
        
        if l1.isdigit() == True and l2.isupper() == True and l3.islower() == True: #Numeros pegados a letras mayusculas 
            pdfFileContent = pdfFileContent.replace(l1+l2+l3,l1+" "+l2+l3)
        
    Tokens_1 = word_tokenize(pdfFileContent) 
    Tokens_vacantes_1 = word_tokenize(requerimientos)
    
    punctuation = re.compile(r'[-+/.?!,:;()•|@[0-9]') #lista de caracteres que removeremos

    Tokens_2 = []
    for words in Tokens_1:
        word = punctuation.sub("",words)
        if len(word)>0:
            Tokens_2.append(word)
        
    Tokens_vacantes_2 = []
    for words in Tokens_vacantes_1:
        word = punctuation.sub("",words)
        if len(word)>0:
            Tokens_vacantes_2.append(word)
            
    stopw = stopwords.words("spanish") #Removemos las palabras de relleno que no aportan informacion
    Tokens_3 = [word for word in Tokens_2 if word not in stopw] 
    Tokens_vacantes_3 = [word for word in Tokens_vacantes_2 if word not in stopw] 
    
    sbst = SnowballStemmer('spanish')

    Tokens_4 = []
    for words in Tokens_3:
        word = sbst.stem(words)
        Tokens_4.append(word)

    dic_revertir = {}
    Tokens_vacantes_4 = []
    for words in Tokens_vacantes_3:
        word = sbst.stem(words)
        Tokens_vacantes_4.append(word)
        dic_revertir[word] = words
        
    sentence_1 = " ".join(Tokens_4)
    sentence_2 = " ".join(Tokens_vacantes_4)
    
    CountVec = CountVectorizer(ngram_range=(1,1)) # to use bigrams ngram_range=(2,2)
    Count_data = CountVec.fit_transform([sentence_1,sentence_2])
    cv_dataframe = pd.DataFrame(Count_data.toarray(),columns = CountVec.get_feature_names_out())
    
    count = 0
    count2 = 0
    matches = []

    for i,j in zip(cv_dataframe.iloc[0],cv_dataframe.iloc[1]):
        if i >= 1 and j >=1:
            count += 1
            matches.append(cv_dataframe.columns.values[count2])
        count2 += 1
        
    final_score = (count / len(Tokens_vacantes_4))*100
    
    final_matches = []
    for i in matches: #lo que se encontro dentro del curriculum
        final_matches.append(dic_revertir[i])
    
    #print(Tokens_vacantes_4)
    
    #print(Tokens_4)
    
    return round(final_score),final_matches
        

In [5]:
requerimientos = vacanteCSV_str("Requisitos_Vacantes.csv","Facturacion")

In [6]:
pdfFileContent = getPDFFileContentToTXT('CVF1.pdf')
os.remove("pdfContenido.txt") #removemos el archivo de texto creado (opcional)

In [49]:
i,j = tokenize1_2_3_4(pdfFileContent,requerimientos)

In [8]:
print(i,j)

65 ['caja', 'clientes', 'cobranza', 'credito', 'dia', 'diario', 'Elaboración', 'Encargado', 'facturación', 'notas', 'prepa', 'saldo', 'telefonica']


# Guardar todo en un Diccionario 

In [139]:
hola = "/Users/andressaldana/Documents/GitHub/Sautek-Natural-language-programming"

archivos  = os.listdir(hola)

In [140]:
arch  = hola + "/" + archivos[0]

In [141]:
dict_final = {}
pdfs = []

for i in archivos: #abre todos los archivos con extension pdf y los guerda en pdfs
        arch = hola + "/" + i 
        if arch[-4:] == ".pdf":
            pdfs.append(arch)
            dict_final[i] = ""
            
            


In [142]:
count = 0 #se asignan los puntajes de los curriculumns 
llaves = list(dict_final.keys())

for file in pdfs:#lista de todos los titulos de los archivos pdfs 
    pdfFileContent = getPDFFileContentToTXT(file)
    os.remove("pdfContenido.txt")
    i_score ,j_palabras = tokenize1_2_3_4(pdfFileContent,requerimientos)
    dict_final[llaves[count]] = [i_score,j_palabras] #guardamos la calificacion y las palabras en el diccionario final 
    count += 1
    
dict_final["CVI1.pdf"]

[10, ['Encargado', 'mostrador']]

In [143]:
x = {"uno": 2, "dos": 4, "tres": 3, "cuatro": 1, "cinco": 0}
dict(sorted(x.items(), key=lambda item: item[1]))


{'cinco': 0, 'cuatro': 1, 'uno': 2, 'tres': 3, 'dos': 4}

In [144]:
dict_final = dict(sorted(dict_final.items(), key=lambda item: item[1], reverse=True))

In [145]:
label3text = "" #se junta toda el diccionario en un solo texto para poder desplegarlo 
llaves = list(dict_final.keys())

for i in llaves:
    label3text += str(i) + " Puntuacion: " + str(dict_final[i][0]) + "% \n"

In [146]:
print(label3text)

CVF1.pdf Puntuacion: 65% 
CVF4.pdf Puntuacion: 60% 
CVF3.pdf Puntuacion: 35% 
CVF2.pdf Puntuacion: 30% 
CVI2.pdf Puntuacion: 15% 
CVI1.pdf Puntuacion: 10% 



# Generar CSV

In [160]:
salida = open("Resultados.csv", mode = "w", newline = "", encoding = "utf-8")

In [161]:
csv_writer = csv.writer(salida, delimiter = ",")

In [128]:
#for i in llaves:
#    csv_writer.writerow([i, " Puntuacion: ", str(dict_final[i][0]) + "%", "Requisitos encontrados: ", " | ".join(dict_final[i][1])]
#                       )

In [162]:
for i in llaves:
    csv_writer.writerow([i, " Puntuacion: ", str(dict_final[i][0]) + "%", "Requisitos encontrados: "] + dict_final[i][1])
   

In [163]:
salida.close()

In [115]:
os.getcwd()

'/Users/andressaldana/Documents/GitHub/Sautek-Natural-language-programming'